In [1]:
import pandas as pd
import import_ipynb

In [2]:
# Function Imports
from word_frequency import word_frequency
from average_named_years import named_years
from year_from_wars import year_from_wars
from Sentiment_Analysis import positivity_score, build_sentiment_model
from Some_functions_Copy import sentlen, wordlen, avesylls, SWprop, readlvl, sentcount, wordcount
from word_pca_Copy import PCAphrases
import nltk
#from word_pca import pseudoPCA
#from word_pca import PCAphrases

importing Jupyter notebook from word_frequency.ipynb
importing Jupyter notebook from average_named_years.ipynb
importing Jupyter notebook from year_from_wars.ipynb
importing Jupyter notebook from Sentiment_Analysis.ipynb
importing Jupyter notebook from Some_functions_Copy.ipynb


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alicekrupczak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


importing Jupyter notebook from word_pca_Copy.ipynb


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alicekrupczak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('twitter_samples')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alicekrupczak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alicekrupczak/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/alicekrupczak/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/alicekrupczak/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alicekrupczak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [56]:
# SVM Imports
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA

## Reading in Data File

In [5]:
orig_data = pd.read_csv("archive/presidential_speeches.csv")

names = ['George Washington', 'Donald Trump'] 
data = orig_data[orig_data['President'].isin(names)]
data = data.reset_index(drop = True)
data.tail()

,Date,President,Party,Speech Title,Summary,Transcript,URL
35,2019-01-19,Donald Trump,Republican,Remarks about the US Southern Border,President Donald Trump speaks about what he se...,"Just a short time ago, I had the honor of pres...",https://millercenter.org/the-presidency/presid...
36,2019-02-05,Donald Trump,Republican,State of the Union Address,"In his second State of the Union Address, Pres...","Madam Speaker, Mr. Vice President, Members of ...",https://millercenter.org/the-presidency/presid...
37,2019-02-15,Donald Trump,Republican,Speech Declaring a National Emergency,President Donald Trump declares a national eme...,"Thank you very much, everybody. Before we begi...",https://millercenter.org/the-presidency/presid...
38,2019-09-24,Donald Trump,Republican,Remarks at the United Nations General Assembly,President Donald Trump speaks to the 74th sess...,"Thank you very much. Mr. President, Mr. Secret...",https://millercenter.org/the-presidency/presid...
39,2019-09-25,Donald Trump,Republican,Press Conference,President Donald Trump holds a press conferenc...,"Thank you very much. Thank you. Well, thank yo...",https://millercenter.org/the-presidency/presid...


## Adding Features to DataFrame

In [6]:
data['Word Frequency'] = word_frequency(data, n = 10, remove_stopwords = True)
data['Named Years'] = named_years(data)
data['Years from Wars'] = year_from_wars(data)

In [7]:
classifier = build_sentiment_model()
data['Positivity Score'] = [positivity_score(x, classifier) for x in data['Transcript']]

In [8]:
data['Sentence Length'] = sentlen(data['Transcript'])
data['Word Length'] = wordlen(data['Transcript'])
data['Syllables per word'] = avesylls(data['Transcript'])
data['Stop Word Proportion'] = SWprop(data['Transcript'])
data['No. of Words'] = wordcount(data['Transcript'])
data['No. of Sentences'] = sentcount(data['Transcript'])

In [9]:
data['Reading Level'] = readlvl(data['Transcript'])

In [10]:
data_PCA = data[['President', 'Transcript']]
PCAfeatures = PCAphrases(data_PCA, n = 1, numfeatures = 10)
PCAfeatures.head()

,0,1,2,3,4,5,6,7,8,9
0,-0.548712,0.275576,-0.567489,-0.027097,0.059906,-0.133182,-0.120437,0.151074,-0.253441,0.801772
1,-0.947810,-0.495632,-0.046101,-0.403872,-0.157283,-0.407563,-0.348685,0.326787,0.204707,-0.091339
2,-0.755548,-0.019439,-0.355541,-0.198894,0.130682,0.061081,-0.242606,0.109844,0.084550,0.072462
3,-0.515819,0.360649,-0.472736,-0.090696,0.395484,0.404612,0.041788,0.105682,0.206166,-0.275426
4,-0.518589,0.044521,-0.575761,-0.232955,-0.178569,0.635761,0.322653,-1.660075,-0.506875,0.308881


In [11]:
for i in range(10):
    data[f'Word PCA {i}'] = PCAfeatures[i]

Now that we have added all of the features to the data frame, this is what we are left with:

In [12]:
data

,Date,President,Party,Speech Title,Summary,Transcript,URL,Word Frequency,Named Years,Years from Wars,...,Word PCA 0,Word PCA 1,Word PCA 2,Word PCA 3,Word PCA 4,Word PCA 5,Word PCA 6,Word PCA 7,Word PCA 8,Word PCA 9
0,1789-04-30,George Washington,Unaffiliated,First Inaugural Address,Washington calls on Congress to avoid local an...,Fellow Citizens of the Senate and the House of...,https://millercenter.org/the-presidency/presid...,0.075556,NaN,NaN,...,-0.548712,0.275576,-0.567489,-0.027097,0.059906,-0.133182,-0.120437,0.151074,-0.253441,0.801772
1,1789-10-03,George Washington,Unaffiliated,Thanksgiving Proclamation,"At the request of Congress, Washington establi...",Whereas it is the duty of all Nations to ackno...,https://millercenter.org/the-presidency/presid...,0.131455,NaN,NaN,...,-0.947810,-0.495632,-0.046101,-0.403872,-0.157283,-0.407563,-0.348685,0.326787,0.204707,-0.091339
2,1790-01-08,George Washington,Unaffiliated,First Annual Message to Congress,"In a wide ranging speech, President Washington...",Fellow Citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...,0.086849,NaN,NaN,...,-0.755548,-0.019439,-0.355541,-0.198894,0.130682,0.061081,-0.242606,0.109844,0.084550,0.072462
3,1790-12-08,George Washington,Unaffiliated,Second Annual Message to Congress,Washington focuses on commerce in his second a...,Fellow citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...,0.072531,NaN,NaN,...,-0.515819,0.360649,-0.472736,-0.090696,0.395484,0.404612,0.041788,0.105682,0.206166,-0.275426
4,1790-12-29,George Washington,Unaffiliated,Talk to the Chiefs and Counselors of the Senec...,The President reassures the Seneca Nation that...,"I the President of the United States, by my ow...",https://millercenter.org/the-presidency/presid...,0.139605,NaN,NaN,...,-0.518589,0.044521,-0.575761,-0.232955,-0.178569,0.635761,0.322653,-1.660075,-0.506875,0.308881
5,1791-10-25,George Washington,Unaffiliated,Third Annual Message to Congress,Washington praises the success of the new bank...,"I meet you, upon the present occasion, with th...",https://millercenter.org/the-presidency/presid...,0.083877,NaN,NaN,...,-0.027352,1.340361,-0.821939,0.305304,0.517219,0.715018,1.149790,-0.569729,-0.618372,-0.437710
6,1792-04-05,George Washington,Unaffiliated,Veto Message on Congressional Redistricting,President Washington returns a congressional r...,Gentlemen of the House of Representatives: I h...,https://millercenter.org/the-presidency/presid...,0.313253,NaN,NaN,...,-1.098004,-0.561390,-0.292741,-0.096594,0.114532,-0.182032,-0.258017,0.175351,-0.003134,0.287104
7,1792-11-06,George Washington,Unaffiliated,Fourth Annual Message to Congress,NaN,"Fellow Citizens of the Senate, and of the Hous...",https://millercenter.org/the-presidency/presid...,0.064091,NaN,NaN,...,-0.048669,1.191001,-0.753820,0.199836,0.743013,0.617804,0.501942,-0.276603,-0.589452,-0.120335
8,1792-12-12,George Washington,Unaffiliated,Proclamation Against Crimes Against the Cherok...,Offering a reward for the capture of American ...,"Whereas I have received authentic information,...",https://millercenter.org/the-presidency/presid...,0.170000,NaN,NaN,...,-1.082036,-0.571434,-0.272996,-0.139497,0.055815,-0.246051,-0.298709,0.138544,0.055583,0.183564
9,1793-03-04,George Washington,Unaffiliated,Second Inaugural Address,"In a simple, brief speech, Washington expresse...",Fellow Citizens: I am again called upon by the...,https://millercenter.org/the-presidency/presid...,0.206349,NaN,NaN,...,-1.113916,-0.588717,-0.343196,-0.111115,0.181908,-0.189242,-0.422160,0.223652,0.136727,0.315664


This cell will save the data frame as a csv. That way, after we have saved it the first time, we won't have to re-run all of the cells involved in creating the data frame every time we start a new kernel.

In [13]:
data.to_csv('features_df.csv')

# NEXT: Machine Learning using Dataframe 

In [49]:
data = pd.read_csv('features_df.csv',index_col=0) # Reloads the data frame from the csv
data 

,Date,President,Party,Speech Title,Summary,Transcript,URL,Word Frequency,Named Years,Years from Wars,...,Word PCA 0,Word PCA 1,Word PCA 2,Word PCA 3,Word PCA 4,Word PCA 5,Word PCA 6,Word PCA 7,Word PCA 8,Word PCA 9
0,1789-04-30,George Washington,Unaffiliated,First Inaugural Address,Washington calls on Congress to avoid local an...,Fellow Citizens of the Senate and the House of...,https://millercenter.org/the-presidency/presid...,0.075556,NaN,NaN,...,-0.548712,0.275576,-0.567489,-0.027097,0.059906,-0.133182,-0.120437,0.151074,-0.253441,0.801772
1,1789-10-03,George Washington,Unaffiliated,Thanksgiving Proclamation,"At the request of Congress, Washington establi...",Whereas it is the duty of all Nations to ackno...,https://millercenter.org/the-presidency/presid...,0.131455,NaN,NaN,...,-0.947810,-0.495632,-0.046101,-0.403872,-0.157283,-0.407563,-0.348685,0.326787,0.204707,-0.091339
2,1790-01-08,George Washington,Unaffiliated,First Annual Message to Congress,"In a wide ranging speech, President Washington...",Fellow Citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...,0.086849,NaN,NaN,...,-0.755548,-0.019439,-0.355541,-0.198894,0.130682,0.061081,-0.242606,0.109844,0.084550,0.072462
3,1790-12-08,George Washington,Unaffiliated,Second Annual Message to Congress,Washington focuses on commerce in his second a...,Fellow citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...,0.072531,NaN,NaN,...,-0.515819,0.360649,-0.472736,-0.090696,0.395484,0.404612,0.041788,0.105682,0.206166,-0.275426
4,1790-12-29,George Washington,Unaffiliated,Talk to the Chiefs and Counselors of the Senec...,The President reassures the Seneca Nation that...,"I the President of the United States, by my ow...",https://millercenter.org/the-presidency/presid...,0.139605,NaN,NaN,...,-0.518589,0.044521,-0.575761,-0.232955,-0.178569,0.635761,0.322653,-1.660075,-0.506875,0.308881
5,1791-10-25,George Washington,Unaffiliated,Third Annual Message to Congress,Washington praises the success of the new bank...,"I meet you, upon the present occasion, with th...",https://millercenter.org/the-presidency/presid...,0.083877,NaN,NaN,...,-0.027352,1.340361,-0.821939,0.305304,0.517219,0.715018,1.149790,-0.569729,-0.618372,-0.437710
6,1792-04-05,George Washington,Unaffiliated,Veto Message on Congressional Redistricting,President Washington returns a congressional r...,Gentlemen of the House of Representatives: I h...,https://millercenter.org/the-presidency/presid...,0.313253,NaN,NaN,...,-1.098004,-0.561390,-0.292741,-0.096594,0.114532,-0.182032,-0.258017,0.175351,-0.003134,0.287104
7,1792-11-06,George Washington,Unaffiliated,Fourth Annual Message to Congress,NaN,"Fellow Citizens of the Senate, and of the Hous...",https://millercenter.org/the-presidency/presid...,0.064091,NaN,NaN,...,-0.048669,1.191001,-0.753820,0.199836,0.743013,0.617804,0.501942,-0.276603,-0.589452,-0.120335
8,1792-12-12,George Washington,Unaffiliated,Proclamation Against Crimes Against the Cherok...,Offering a reward for the capture of American ...,"Whereas I have received authentic information,...",https://millercenter.org/the-presidency/presid...,0.170000,NaN,NaN,...,-1.082036,-0.571434,-0.272996,-0.139497,0.055815,-0.246051,-0.298709,0.138544,0.055583,0.183564
9,1793-03-04,George Washington,Unaffiliated,Second Inaugural Address,"In a simple, brief speech, Washington expresse...",Fellow Citizens: I am again called upon by the...,https://millercenter.org/the-presidency/presid...,0.206349,NaN,NaN,...,-1.113916,-0.588717,-0.343196,-0.111115,0.181908,-0.189242,-0.422160,0.223652,0.136727,0.315664


## SVM

### Imputing the Data

To make things simpler, I'm deleting all non-numerical columns and changing the Presidential labels to numbers.

In [50]:
data = data.drop(columns = ['Date', 'Party', 'Speech Title', 'Summary', 'Transcript', 'URL'], axis = 1)

In [51]:
presidents_to_classify = pd.unique(data['President'])

# The exact value of the president will vary based on how many presidents we are trying to classify.
for p in range(0, len(presidents_to_classify)):
    for i in range(0, len(data['President'])):
        if data['President'][i] == presidents_to_classify[p]:
            data['President'][i] = p

<ipython-input-51-e489914c8dd6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['President'][i] = p


Replace any missing data with the column means, for now.

In [52]:
data['Named Years'].fillna(value=data['Named Years'].mean(), inplace=True)
data['Years from Wars'].fillna(value=data['Years from Wars'].mean(), inplace=True)

### Separate the Features from the Labels

In [72]:
features = data.drop(columns = 'President')
labels = data['President'].astype(int)
features.head()

,Word Frequency,Named Years,Years from Wars,Positivity Score,Sentence Length,Word Length,Syllables per word,Stop Word Proportion,No. of Words,No. of Sentences,...,Word PCA 0,Word PCA 1,Word PCA 2,Word PCA 3,Word PCA 4,Word PCA 5,Word PCA 6,Word PCA 7,Word PCA 8,Word PCA 9
0,0.075556,0.994348,0.978669,0.791667,59.750000,4.943515,1.596234,0.544630,1434,24,...,-0.548712,0.275576,-0.567489,-0.027097,0.059906,-0.133182,-0.120437,0.151074,-0.253441,0.801772
1,0.131455,0.994348,0.978669,1.000000,108.750000,4.751724,1.531034,0.528736,435,4,...,-0.947810,-0.495632,-0.046101,-0.403872,-0.157283,-0.407563,-0.348685,0.326787,0.204707,-0.091339
2,0.086849,0.994348,0.978669,0.761905,40.380952,5.128538,1.656840,0.534198,848,21,...,-0.755548,-0.019439,-0.355541,-0.198894,0.130682,0.061081,-0.242606,0.109844,0.084550,0.072462
3,0.072531,0.994348,0.978669,0.825000,35.025000,4.893647,1.538187,0.548894,1401,40,...,-0.515819,0.360649,-0.472736,-0.090696,0.395484,0.404612,0.041788,0.105682,0.206166,-0.275426
4,0.139605,0.994348,0.978669,0.720000,29.208333,4.513552,1.390157,0.545649,1402,48,...,-0.518589,0.044521,-0.575761,-0.232955,-0.178569,0.635761,0.322653,-1.660075,-0.506875,0.308881


### Split the Data into Test and Training Sets

Initially, the training set is set to be 75% of the data.

In [74]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, train_size = 0.75, random_state = 0)

### Finding the Best Parameters for Model Using Grid Search

In [81]:
param_grid = {'kernel':('linear', 'rbf'),
              'C': [1, 10, 100, 1000],
              'gamma': [0.0001, 0.001, 0.01, 0.1],}

clf = GridSearchCV(SVC(), param_grid)

clf = clf.fit(train_features, train_labels)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Best estimator found by grid search:
SVC(C=1, gamma=0.0001, kernel='linear')


In [82]:
pred_labels = clf.predict(test_features)

conf_mat = confusion_matrix(test_labels, pred_labels)
class_rep = classification_report(test_labels, pred_labels)

print(f'Confusion Matrix: \n {conf_mat}')
print(class_rep)

Confusion Matrix: 
 [[5 1]
 [0 4]]
              precision    recall  f1-score   support

           0       1.00      0.83      0.91         6
           1       0.80      1.00      0.89         4

    accuracy                           0.90        10
   macro avg       0.90      0.92      0.90        10
weighted avg       0.92      0.90      0.90        10

